## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from os.path import expanduser

___
### Add to sys.path

Within the barchart_api module, references like:

```from barchartapi import barchart_api```

will not work

In [2]:
WORKSPACE_DIR = '../..' 
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' 
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' 
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
import barchart_api as bcapi

___
### Create paths to where you will store the history data

In [3]:
STOCKS_DIR = f'./temp_folder/stocks/daily'
try:
    os.makedirs(STOCKS_DIR)
except Exception as e:
    pass

___
### Create an instance of BcHist
1. select an enpoint prefix of free or paid
2. must set bar_type to minutes, hour or daily
3. must set interval
 * for minutes, set interval to 1, 5, 15, 30 or 60
 * for other types, see https://www.barchart.com/ondemand/api/getHistory

In [4]:
# select and endpoint prefix
endpoint_prefix='free' # 'free' or 'paid'

# select a bartype and interval
# bar_type='minutes'
# interval=1
# bar_type='daily'
# interval=1
bar_type='minutes'
interval=30

# create an instance of BcHist
api_key = open(f'./temp_folder/{endpoint_prefix}_api_key.txt','r').read()
endpoint_type=f'{endpoint_prefix}_url'
bch = bcapi.BcHist(api_key, bar_type=bar_type, interval=interval,endpoint_type = endpoint_type)


Barchart OnDemand Client: https://marketdata.websol.barchart.com/


___
### Main Loop to fetch history data from Barchart

#### Inputs
On each call to BcHist.get_history, provide:
 * a short name like:
  * a stock symbol like: SPY,USO,IBM, etc
  * a futures symbol like: CLJ18, GCG19, ESH17, etc
 * a beginning yyyymmdd integer
 * an ending yyyymmdd integer

#### Outputs
The get_history method will return a tuple, where:
 * tup[0] provides a status
 * tup[1] provides a pandas DataFrame of data, or None

In [5]:

spydr_short_names = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF']
commodity_etf_short_names = ['USO','UNG','DBC','DBA','GLD','USCI']
currency_etf_short_names = ['FXY','FXE','FXB','FXF','FXC','FXA']
my_portfolio_short_names = ['XLE','SPY','USO','GLD','XLU']
short_names = spydr_short_names
beg_yyyymmdd = 20001201
end_yyyymmdd = 20190112

for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    if os.path.isfile(sn_path):
        print(f'BYPASSING: {short_name}')
        continue
    print(f'get_history: {short_name} BEGIN {datetime.datetime.now()}')
    tup = bch.get_history(short_name, beg_yyyymmdd, end_yyyymmdd)
    print(f'get_history: {short_name} WRITING DATA {datetime.datetime.now()}')
    tup[1].to_csv(sn_path,index=False)
    print(f'get_history: {short_name} END {datetime.datetime.now()}')
    

get_history: XLE BEGIN 2019-01-29 10:58:22.296504
get_history: XLE WRITING DATA 2019-01-29 10:58:22.796054
get_history: XLE END 2019-01-29 10:58:22.799654
get_history: XLU BEGIN 2019-01-29 10:58:22.800051
get_history: XLU WRITING DATA 2019-01-29 10:58:23.900678
get_history: XLU END 2019-01-29 10:58:23.903496
get_history: XLK BEGIN 2019-01-29 10:58:23.903592
get_history: XLK WRITING DATA 2019-01-29 10:58:24.177436
get_history: XLK END 2019-01-29 10:58:24.179721
get_history: XLB BEGIN 2019-01-29 10:58:24.179984
get_history: XLB WRITING DATA 2019-01-29 10:58:24.469181
get_history: XLB END 2019-01-29 10:58:24.471582
get_history: XLP BEGIN 2019-01-29 10:58:24.471643
get_history: XLP WRITING DATA 2019-01-29 10:58:24.773250
get_history: XLP END 2019-01-29 10:58:24.775702
get_history: XLY BEGIN 2019-01-29 10:58:24.776019
get_history: XLY WRITING DATA 2019-01-29 10:58:25.046961
get_history: XLY END 2019-01-29 10:58:25.049411
get_history: XLI BEGIN 2019-01-29 10:58:25.049479
get_history: XLI WRI

___
### Do something interesting with the returned data

In [6]:
df_portfolio = None
for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    df_temp = pd.read_csv(sn_path)
    df_temp = df_temp[['timestamp','close']]
    df_temp = df_temp.rename(columns={'close':f'{short_name}'})
    if df_portfolio is None:
        df_portfolio = df_temp.copy()
    else:
        df_portfolio = df_portfolio.merge(df_temp,how='inner',on='timestamp')
df_corr = df_portfolio.corr()
df_corr

,XLE,SPY,USO,GLD,XLU
XLE,1.000000,0.969503,0.955771,-0.172805,0.568791
SPY,0.969503,1.000000,0.893435,-0.297334,0.675616
USO,0.955771,0.893435,1.000000,-0.159030,0.532540
GLD,-0.172805,-0.297334,-0.159030,1.000000,-0.835103
XLU,0.568791,0.675616,0.532540,-0.835103,1.000000


In [7]:
fn = f'{STOCKS_DIR}/{"_".join(df_corr.columns.values)}.csv'
df_corr.to_csv(fn,index=False)

In [8]:
df_portfolio.head()

,timestamp,XLE,SPY,USO,GLD,XLU
0,2018-12-12T09:30:00-05:00,63.49605,265.5294,11.100,117.8200,56.37121
1,2018-12-12T10:00:00-05:00,63.49605,265.9470,11.040,117.8979,56.38113
2,2018-12-12T10:30:00-05:00,63.64466,266.5534,11.065,117.8346,56.26213
3,2018-12-12T11:00:00-05:00,63.74374,266.3944,11.100,117.7700,56.14314
4,2018-12-12T11:30:00-05:00,63.64466,266.8020,11.060,117.7500,56.11340


In [9]:
cols = list(set(list(df_portfolio.columns.values))-set(['timestamp']))
bars_per_day = 8*2
perc_of_day = 1/bars_per_day
perc_of_year = perc_of_day/256
df_portfolio[cols].pct_change().iloc[1:].std()/perc_of_year**.5

GLD    0.119333
XLE    0.349473
USO    0.517850
SPY    0.282906
XLU    0.233001
dtype: float64

## END